Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [7]:
import warnings

warnings.filterwarnings('ignore')

In [8]:
# Импорт библиотек
import numpy as np # для матричных вычислений
import pandas as pd # для анализа и предобработки данных

from sklearn import linear_model # линейные моделиё
from sklearn import tree # деревья решений
from sklearn import ensemble # ансамбли
from sklearn import metrics # метрики
from sklearn import preprocessing # предобработка
from sklearn.model_selection import train_test_split # сплитование выборки

# Импорт методов подбора гиперпараметров
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

In [9]:
# Загружаем и смотрим датасет
data = pd.read_csv('data/_train_sem09.csv')
data.head(3)

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0


## Модель ##

In [59]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

## Логическая регресия

In [91]:
# Создаем объект класса логистическая регресси
log_reg = linear_model.LogisticRegression(max_iter = 50, random_state = 20)

# Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)

# Выводим результат по метрике f1
y_train_pred_log = log_reg.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred_log)))
y_test_pred_log = log_reg.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_log)))

Train: 0.87
Test: 0.79


GridSeachCV

In [95]:
# Словарь с именами гиперпараметров 
param_grid = [{'penalty': ['l2', 'none'] ,#  тип регурялизации
              'solver': ['lbfgs', 'saga'], # алгоритм оптимизации
              'C': [0.01, 0.1, 0.3, 0.5]} # уровень силы регурялизации
              ]

# Вызваем класс GridSearchCV и передаем модель LogisticRegression, сетку искомых параметров
grid_search_log = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=20, # генератор случайных чисел
        max_iter=50 # количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, # число фолдов
    n_jobs = -1 # используем все доступные ядра для расчётов
)  

# Вызываем объект grid_search_log стандартные методы fit, который запускает кросс-валидацию для каждой комбинации гиперпараметров
# %time определяет время работы
%time grid_search_log.fit(X_train, y_train) 

# Выводим результат по метрике f1
y_train_pred = grid_search_log.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = grid_search_log.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

CPU times: total: 2.45 s
Wall time: 2min 19s
Train: 0.85
Test: 0.79


In [96]:
print("Наилучшее значение точности при кросс-валидаци: {:.2f}".format(grid_search_log.best_score_))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_log.best_params_))
print("Наилучшая модель:\n{}".format(grid_search_log.best_estimator_))

Наилучшее значение точности при кросс-валидаци: 0.76
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Наилучшая модель:
LogisticRegression(C=0.1, max_iter=50, random_state=20)


In [97]:
# Взглянем на результаты кросс-валидации
result_cv1 = pd.DataFrame(grid_search_log.cv_results_)
result_cv1.head(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.024342,0.649268,0.107206,0.031436,0.01,l2,lbfgs,"{'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}",0.736667,0.765,0.780000,0.745000,0.746667,0.754667,0.015684,10
1,11.326043,0.824789,0.092811,0.034170,0.01,l2,saga,"{'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}",0.738333,0.760,0.778333,0.746667,0.745000,0.753667,0.014197,11


RandomizedSearchCV

In [63]:
# Создаем словарь
param_distributions = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
              'C': [0.01, 0.1, 0.3, 0.5] # уровень силы регурялизации
               },
# Вызваем класс RandomizedSearchCV и передаем модель LogisticRegression, сетку искомых параметров            
random_search_log = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=20, max_iter=50), 
    param_distributions=param_distributions, 
    cv=5, # количество фолдов
    n_iter = 10, # количество комбинаций на расчёт
    n_jobs = -1 # используем все доступные ядра для расчётов
) 

# Вызываем объект random_search_log стандартные методы fit, который запускает кросс-валидацию для каждой комбинации гиперпараметров
# %time определяет время работы
%time random_search_log.fit(X_train, y_train) 

# Выводим результат по метрике f1
y_train_pred = random_search_log.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = random_search_log.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

CPU times: total: 6.98 s
Wall time: 1min 27s
Train: 0.85
Test: 0.79


In [64]:
print("Наилучшее значение точности при кросс-валидаци: {:.2f}".format(random_search_log.best_score_))
print("Наилучшие значения гиперпараметров: {}".format(random_search_log.best_params_))
print("Наилучшая модель:\n{}".format(random_search_log.best_estimator_))

Наилучшее значение точности при кросс-валидаци: 0.76
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.1}
Наилучшая модель:
LogisticRegression(C=0.1, max_iter=50, random_state=20, solver='sag')


In [65]:
# Взглянем на результаты кросс-валидации
result_cv2 = pd.DataFrame(random_search_log.cv_results_)
result_cv2.head(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,param_penalty,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.394604,0.496094,0.086221,0.015502,lbfgs,none,0.1,"{'solver': 'lbfgs', 'penalty': 'none', 'C': 0.1}",0.71,0.735000,0.733333,0.731667,0.740000,0.730000,0.010382,9
1,8.315866,0.103308,0.073220,0.010987,sag,l2,0.1,"{'solver': 'sag', 'penalty': 'l2', 'C': 0.1}",0.73,0.761667,0.775000,0.763333,0.763333,0.758667,0.015107,1


## Случайный лес

In [68]:
# Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=20)

# Обучаем модель
rf.fit(X_train, y_train)

# Выводим результат по метрике f1
y_train_pred_forest = rf.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred_forest)))
y_test_pred_forest = rf.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_forest)))

Train: 1.00
Test: 0.82


RandomizedSearchCV

In [69]:
# Создаем словарь
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 10, dtype=int))
              }

# Вызваем класс RandomizedSearchCV и передаем модель RandomForestClassifier, сетку искомых параметров            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=20), 
    param_distributions=param_distributions, 
    cv=5, # количество фолдов
    n_iter = 10, # количество комбинаций на расчёт
    n_jobs = -1 # используем все доступные ядра для расчётов
)

# Вызываем объект random_search_fores стандартные методы fit, который запускает кросс-валидацию для каждой комбинации гиперпараметров 
# %time определяет время работы 
%time random_search_forest.fit(X_train, y_train) 

# Выводим результат по метрике f1
y_train_pred = random_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

CPU times: total: 4.67 s
Wall time: 1min 37s
f1_score на обучающем наборе: 0.94
f1_score на тестовом наборе: 0.82


In [70]:
print("Наилучшее значение точности при кросс-валидаци: {:.2f}".format(random_search_forest.best_score_))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))
print("Наилучшая модель:\n{}".format(random_search_forest.best_estimator_))

Наилучшее значение точности при кросс-валидаци: 0.79
Наилучшие значения гиперпараметров: {'n_estimators': 110, 'min_samples_leaf': 5, 'max_depth': 20}
Наилучшая модель:
RandomForestClassifier(max_depth=20, min_samples_leaf=5, n_estimators=110,
                       random_state=20)


In [71]:
# Взглянем на результаты кросс-валидации
result_cv2 = pd.DataFrame(random_search_forest.cv_results_)
result_cv2.head(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_leaf,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,7.211409,0.096853,0.282074,0.011545,140,5,24,"{'n_estimators': 140, 'min_samples_leaf': 5, '...",0.771667,0.801667,0.796667,0.791667,0.775000,0.787333,0.011907,2
1,5.508789,0.151931,0.230261,0.030649,110,5,20,"{'n_estimators': 110, 'min_samples_leaf': 5, '...",0.780000,0.796667,0.796667,0.791667,0.778333,0.788667,0.007986,1


GridSeachCV

In [73]:
# Словарь с именами гиперпараметров 
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 10, dtype=int))
              }

# Вызваем класс GridSearchCV и передем модель RandomForestClassifier, сетку искомых параметров            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=20), 
    param_grid=param_grid, 
    cv=5,  # число фолдов
    n_jobs = -1 # используем все доступные ядра для расчётов
)  

# Вызываем объект grid_search_forest стандартные методы fit, который запускает кросс-валидацию для каждой комбинации гиперпараметров
# %time определяет время работы
%time grid_search_forest.fit(X_train, y_train) 

# Выводим результат по метрике f1
y_train_pred = grid_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = grid_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

CPU times: total: 6.39 s
Wall time: 6min 32s
f1_score на обучающем наборе: 0.94
f1_score на тестовом наборе: 0.82


In [74]:
print("Наилучшее значение точности при кросс-валидаци: {:.2f}".format(grid_search_forest.best_score_))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))
print("Наилучшая модель:\n{}".format(grid_search_forest.best_estimator_))

Наилучшее значение точности при кросс-валидаци: 0.79
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 110}
Наилучшая модель:
RandomForestClassifier(max_depth=20, min_samples_leaf=5, n_estimators=110,
                       random_state=20)


In [75]:
# Взглянем на результаты кросс-валидации
result_cv2 = pd.DataFrame(grid_search_forest.cv_results_)
result_cv2.head(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.545094,0.041392,0.179651,0.013894,20,5,80,"{'max_depth': 20, 'min_samples_leaf': 5, 'n_es...",0.766667,0.786667,0.795000,0.788333,0.776667,0.782667,0.009922,32
1,4.971650,0.079732,0.189852,0.011639,20,5,110,"{'max_depth': 20, 'min_samples_leaf': 5, 'n_es...",0.780000,0.796667,0.796667,0.791667,0.778333,0.788667,0.007986,1


Hyperopt

In [77]:
# Зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

# Зафксируем random_state
random_state = 20

# Создаем функцию для минимизации
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # Функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
    # Используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    # обучаем модель
    model.fit(X, y)
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [78]:
%time # определяет время работы
# Начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state) # фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

# Рассчитаем точность для тестовой выборки
model_rt = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model_rt.fit(X_train, y_train)

# Выводим результат по метрике f1
y_train_pred = model_rt.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model_rt.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

CPU times: total: 0 ns
Wall time: 0 ns
100%|██████████| 50/50 [18:18<00:00, 21.96s/trial, best loss: -0.812564892112643]
Наилучшие значения гиперпараметров {'max_depth': 20.0, 'min_samples_leaf': 3.0, 'n_estimators': 175.0}
Train: 0.98
Test: 0.82


Optuna

In [79]:
# Создаем функцию максимализации
def optuna_rf(trial):
    #Задаем пространства поиска гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
    max_depth = trial.suggest_int('max_depth', 15, 26, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

    # Создаем модель
    model_optu = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
    # Обучаем модель
    model_optu.fit(X_train, y_train)
    # Применим  cross validation
    score = cross_val_score(model_optu, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

    return score

In [80]:
%time # определяет время работы
# Создаем объект исследования
# Напрямую указываем, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# Ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

# Выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

# Рассчитаем точность для тестовой выборки
model_opt = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model_opt.fit(X_train, y_train)

# Выводим результат по метрике f1
y_train_pred = model_opt.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model_opt.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

CPU times: total: 0 ns
Wall time: 0 ns


[I 2023-06-15 19:12:12,058] A new study created in memory with name: RandomForestClassifier
[I 2023-06-15 19:12:32,274] Trial 0 finished with value: 0.8054450826888327 and parameters: {'n_estimators': 117, 'max_depth': 18, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8054450826888327.
[I 2023-06-15 19:12:47,418] Trial 1 finished with value: 0.8041592977270566 and parameters: {'n_estimators': 107, 'max_depth': 19, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.8054450826888327.
[I 2023-06-15 19:13:14,717] Trial 2 finished with value: 0.8095265348311695 and parameters: {'n_estimators': 190, 'max_depth': 16, 'min_samples_leaf': 5}. Best is trial 2 with value: 0.8095265348311695.
[I 2023-06-15 19:13:34,575] Trial 3 finished with value: 0.8096446833834484 and parameters: {'n_estimators': 131, 'max_depth': 23, 'min_samples_leaf': 4}. Best is trial 3 with value: 0.8096446833834484.
[I 2023-06-15 19:13:58,184] Trial 4 finished with value: 0.8020436054561655 and parameters: {'n_

Наилучшие значения гиперпараметров {'n_estimators': 149, 'max_depth': 24, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.82
Train: 0.99
Test: 0.83


In [100]:
# pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
